In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [16]:
from category_encoders.target_encoder import TargetEncoder
from category_encoders.utils import *
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV,StratifiedKFold,KFold
from sklearn.metrics import *

train,validate=train_test_split(df1,test_size=0.3,random_state=123)

def k_fold_target(train,test,columns,target,folds=10):
    kf=KFold()
    t=train
    v=test
    global_mean=t[target].mean()
    for col in columns:
        #t[col+'_enc']=np.nan
        #v[col+'_enc']=np.nan
        for i,j in kf.split(t):
            #tr,vl=t.iloc[i],t.iloc[j]
            m=t.loc[t.index[i]].groupby(col)[target].mean()
            t.loc[t.index[j],col+'_enc']=t.loc[t.index[j],col].map(m)
            #print(t[col+'_enc'].isnull().sum())
        t[col+'_enc']=t[col+'_enc'].fillna(global_mean)
        v[col+'_enc']=v[col].map(t.groupby(col)[target].mean())
        v[col+'_enc']=v[col+'_enc'].fillna(global_mean)
        #print(t[col+'_enc'].isnull().sum())
    t=t.drop(columns,axis=1)
    v=v.drop(columns,axis=1)

    return t,v
        

In [11]:
cols=list(df1.columns)
target='target'
cols.remove('target')
t,v=k_fold_target(train,validate,cols,target)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [12]:
t.head()

,target,RESOURCE_enc,MGR_ID_enc,ROLE_ROLLUP_1_enc,ROLE_ROLLUP_2_enc,ROLE_DEPTNAME_enc,ROLE_FAMILY_DESC_enc,ROLE_CODE_enc
7693,1,1.000000,0.636364,0.949079,0.969529,0.954839,0.936000,0.963718
17676,1,0.777778,1.000000,0.863014,0.863014,0.917073,0.966102,0.965596
1969,1,1.000000,1.000000,0.948655,0.962687,0.846154,0.966527,1.000000
15050,1,0.942192,1.000000,0.949079,0.969529,0.974359,1.000000,0.872093
345,0,1.000000,1.000000,0.949079,0.911504,0.933333,0.600000,0.750000


In [13]:
x_tr=t.drop('target',axis=1)
y_tr=t['target']
x_ts=v.drop('target',axis=1)
y_ts=v['target']

In [14]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    print('AUC_ROC= ',auc)
    print('Accuracy= ',acc)
    print('confusion matrix:\n',confusion_matrix(y,pred))
    

In [17]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier()
params={'num_leaves': st.randint(10,50),
    'max_depth':st.randint(3,30) ,
    'learning_rate':st.uniform(0,1) ,
    'n_estimators':st.randint(100,250)}
sf=StratifiedKFold()
rs=RandomizedSearchCV(lgb,param_distributions=params,cv=10,scoring='roc_auc',random_state=1234)
#m=rs.fit(x_tr,y_tr)
lbp=m.best_params_
print('RS:',m.best_score_)
print('params: ',lbp)
lgb=LGBMClassifier(**lbp)
mod=lgb.fit(x_tr,y_tr)
print('\nTrain:')
aucroc(mod,x_tr,y_tr)
print('\nTest:')
aucroc(mod,x_ts,y_ts)


RS: 0.8433141680339915
params:  {'learning_rate': 0.022143913603027254, 'max_depth': 9, 'n_estimators': 239, 'num_leaves': 10}

Train:
AUC_ROC=  0.8775978354296772
Accuracy=  0.948339000784724
confusion matrix:
 [[  228  1098]
 [   87 21525]]

Test:
AUC_ROC=  0.8536302875104493
Accuracy=  0.9447665547757095
confusion matrix:
 [[  74  497]
 [  46 9214]]


In [18]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

mod=lr.fit(x_tr,y_tr)
print('\nTrain:')
aucroc(mod,x_tr,y_tr)
print('\nTest:')
aucroc(mod,x_ts,y_ts)


Train:
AUC_ROC=  0.8271840730625883
Accuracy=  0.9421484000348767
confusion matrix:
 [[  158  1168]
 [  159 21453]]

Test:
AUC_ROC=  0.8371616050050498
Accuracy=  0.9430373308920761
confusion matrix:
 [[  68  503]
 [  57 9203]]


In [89]:
for i in x_tr.columns:
    for j in x_tr[i]:
        if (j==np.nan)or (j==(-np.inf)):
            print(j)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22938 entries, 0 to 22937
Data columns (total 7 columns):
RESOURCE_enc            22938 non-null float64
MGR_ID_enc              22938 non-null float64
ROLE_ROLLUP_1_enc       22938 non-null float64
ROLE_ROLLUP_2_enc       22938 non-null float64
ROLE_DEPTNAME_enc       22938 non-null float64
ROLE_FAMILY_DESC_enc    22938 non-null float64
ROLE_CODE_enc           22938 non-null float64
dtypes: float64(7)
memory usage: 1.2 MB
